In [813]:
from environment import Environment
from magiccube import Cube
from agents import A2C, DQN, PPO
import yaml
import numpy as np
import itertools
import torch
import torch.nn as nn

In [814]:
args = yaml.safe_load(open("config.yaml", "r"))

env = Environment(
    method="LBL",
    size=3,
    args=args["environment"]
)

agent = DQN(env, args["DQN"])

In [815]:
cube = Cube()
print(cube)

          W  W  W                   
          W  W  W                   
          W  W  W                   
 O  O  O  G  G  G  R  R  R  B  B  B 
 O  O  O  G  G  G  R  R  R  B  B  B 
 O  O  O  G  G  G  R  R  R  B  B  B 
          Y  Y  Y                   
          Y  Y  Y                   
          Y  Y  Y                   



In [816]:
env.scramble()
print(env.state2)
print(env.state2.shape)

[[[ 0  4  3  0 17]
  [ 5  0 -1  1 22]
  [ 3  0  5  2 19]
  [ 5 -1  3  3 11]
  [ 1 -1 -1  4  4]
  [ 1 -1  4  5  3]
  [ 1  5  2  6  8]
  [ 2  0 -1  7 24]
  [ 4  2  1  8  6]
  [-1  4  3  9  9]
  [-1  3 -1 10 10]
  [-1  5  1 11  5]
  [-1 -1  4 12 12]
  [-1 -1  5 13 13]
  [-1  0  4 14 20]
  [-1  2 -1 15 15]
  [-1  1  3 16  1]
  [ 2  5  0 17 25]
  [ 0  3 -1 18 18]
  [ 0  2  4 19 23]
  [ 2 -1  5 20 16]
  [ 0 -1 -1 21 21]
  [ 1 -1  2 22  7]
  [ 3  4  1 23  0]
  [ 4  2 -1 24 14]
  [ 5  1  3 25  2]]]
(1, 26, 5)


In [ ]:
hidden_size = 128

embedding = nn.Linear(5, hidden_size)

attention = nn.MultiheadAttention(
    embed_dim=hidden_size,
    num_heads=4,
    batch_first=True
)

layer_norm = nn.LayerNorm(hidden_size)

ffn = nn.Sequential(
    nn.Linear(hidden_size, hidden_size),
    nn.ReLU(inplace=True)
)

output_layer = nn.Sequential(
    nn.Linear(hidden_size * 26, hidden_size * 2),
    nn.ReLU(inplace=True),
    nn.Linear(hidden_size * 2, 12)
)


In [818]:
state_tensor = torch.from_numpy(env.state2).float()
embedded = embedding(state_tensor)
attn_output, _ = attention(embedded, embedded, embedded)
attn_output = layer_norm(embedded + attn_output)
ffn_output = ffn(attn_output)
output = layer_norm(attn_output + ffn_output)

flattened = output.view(1, -1)
output = output_layer(flattened)

output

tensor([[ 0.0094,  0.0012, -0.1942, -0.1318,  0.1543,  0.3571, -0.2437,  0.1882,
         -0.1059, -0.2836, -0.3673, -0.0944]], grad_fn=<AddmmBackward0>)

In [819]:
# Definiamo un layer lineare: input con 3 feature, output con 2 feature
fc1 = nn.Linear(in_features=3, out_features=256)
fc2 = nn.Linear(in_features=256, out_features=2)

# Esempio di input: batch di 4 esempi, ciascuno con 3 feature
input_matrix = torch.tensor([
    [1.0, 2.0, 3.0],
    [4.0, 5.0, 6.0],
    [7.0, 8.0, 9.0],
    [10.0, 11.0, 12.0]
])  # shape: (4, 3)

# Passaggio della matrice al layer
output = fc1(input_matrix)
print(output)
output = fc2(output)

print("Output:", output)

tensor([[-0.5347,  2.1434,  1.0452,  ..., -0.6848,  0.3639, -0.0204],
        [-0.2298,  3.9216,  1.2248,  ..., -1.2748,  1.0381,  0.4056],
        [ 0.0752,  5.6998,  1.4044,  ..., -1.8648,  1.7124,  0.8316],
        [ 0.3801,  7.4780,  1.5840,  ..., -2.4547,  2.3867,  1.2577]],
       grad_fn=<AddmmBackward0>)
Output: tensor([[0.8043, 0.2137],
        [2.7549, 1.3972],
        [4.7054, 2.5807],
        [6.6560, 3.7641]], grad_fn=<AddmmBackward0>)
